In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

LC Medium: Longest Substring with atleast K repeating characters

In [5]:
# https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/
# https://leetcode.com/problems/longest-substring-with-at-least-k-repeating-characters/submissions/1286506890
def longestSubstring(s: str, k: int) -> int:
    """
    Count the freq of all characters, split on the characters with freq < k
    """
    N = len(s)

    # Count the freq
    freq: collections.Counter = collections.Counter(s)
    is_valid = not any(map(lambda x: x < k, freq.values()))

    # If the entire string is okay, return length as answer
    if is_valid:
        return N

    # Split on the invalid characters and backtrack inside
    else:
        max_length = start_idx = 0
        for curr_idx in range(N):
            if 0 < freq[s[curr_idx]] < k:
                max_length = max(max_length, longestSubstring(s[start_idx:curr_idx], k))
                start_idx = curr_idx + 1

        max_length = max(max_length, longestSubstring(s[start_idx:], k))
        return max_length

# Tesitng the solution
assert longestSubstring("aaabb", 3) == 3
assert longestSubstring("ababbc", 2) == 5
assert longestSubstring("bbaaa", 3) == 3

LC Virtual - 4: Weekly Contest 5
https://leetcode.com/contest/leetcode-weekly-contest-5/

In [6]:
# Q4: Frog Jump
def canCrossRecursive(stones: list[int]) -> bool:
    @functools.cache
    def backtrack(n: int, jump: int) -> bool:
        if jump < 0:
            return False
        elif n == stones[-1]:
            return True
        else:
            if n + jump - 1 in positions and backtrack(n + jump - 1, jump - 1):
                return True
            elif jump > 0 and n + jump in positions and backtrack(n + jump, jump):
                return True
            elif n + jump + 1 in positions and backtrack(n + jump + 1, jump + 1):
                return True
            else:
                return False

    N = len(stones)
    positions: dict[int, int] = {n: i for i, n in enumerate(stones)}
    return len(stones) <= 1 or backtrack(stones[0], 0)

# Testing the solution
assert canCrossRecursive([0,1,3,5,6,8,12,17]) == True
assert canCrossRecursive([0,1,2,3,4,8,9,11]) == False
assert canCrossRecursive([]) == True
assert canCrossRecursive([100]) == True
assert canCrossRecursive([0,2]) == False

In [7]:
def canCrossTabulation(stones: list[int]) -> bool:
    N = len(stones)
    positions: dict[int, int] = {n: i for i, n in enumerate(stones)}
    dp: list[list[bool]] = [[False if stones[i] != stones[-1] else True for jump in range(N + 2)] for i in range(N)]
    for i in range(N - 2, -1, -1):
        for jump in range(N):
            n = stones[i]
            if n + jump - 1 in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump - 1]][jump - 1]
            if jump > 0 and n + jump in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump]][jump]
            if n + jump + 1 in positions:
                dp[i][jump] = dp[i][jump] or dp[positions[n + jump + 1]][jump + 1]

    return len(stones) <= 1 or dp[0][0]

# Testing the solution
assert canCrossTabulation([0,1,3,5,6,8,12,17]) == True
assert canCrossTabulation([0,1,2,3,4,8,9,11]) == False
assert canCrossTabulation([]) == True
assert canCrossTabulation([100]) == True
assert canCrossTabulation([0,2]) == False

In [8]:
# Q3: Remove K digits
def removeKdigits(num: str, k: int) -> str:
    pass

In [9]:
# Q1: Find Nth Digit
def findNthDigit(N: int) -> int:
    def countDigitsBefore(n: int) -> int:
        if n < 10:
            return n
        else:
            n_digits = math.floor(math.log10(n)) + 1
            smallest_n_digit_number = 10 ** (n_digits - 1)
            n_digits_before = (n - smallest_n_digit_number + 1) * n_digits
            return n_digits_before + countDigitsBefore(smallest_n_digit_number - 1)

    # Binary search
    low, high = 1, N
    while low <= high:
        mid = (low + high) // 2
        count = countDigitsBefore(mid)
        if count == N:
            return mid % 10
        elif count < N:
            low = mid + 1
        else:
            high = mid - 1

    delta = countDigitsBefore(low) - N
    return int(str(low)[-delta-1])

# Testing the solution
assert findNthDigit(11) == 0
assert findNthDigit(3) == 3

In [10]:
# Q2: Binary Watch
def readBinaryWatch(turnedOn: int) -> set[str]:
    watch: list[bool] = [False for i in range(10)]
    result: set[str] = set()
    def backtrack(lights: int) -> None:
        if lights == 0:
            hour = minute = 0
            for i in range(10):
                if watch[i]:
                    if i < 4:
                        hour += 2 ** (3 - i)
                    else:
                        minute += 2 ** (9 - i)

            if 0 <= hour < 12 and 0 <= minute <= 59:
                result.add(f"{hour}:{str(minute).zfill(2)}")

        else:
            for i in range(10):
                if not watch[i]:
                    watch[i] = True
                    backtrack(lights - 1)
                    watch[i] = False

    backtrack(turnedOn)
    return result

# Testing the solution
assert sorted(readBinaryWatch(1)) == sorted(["0:01","0:02","0:04","0:08","0:16","0:32","1:00","2:00","4:00","8:00"])

Minimum no of moves to seat everyone 
https://leetcode.com/problems/minimum-number-of-moves-to-seat-everyone

In [11]:
def minMovesToSeat(seats: list[int], students: list[int]) -> int:
    # Compute length
    N = len(seats)

    # Sort both to ensure that we get the minimal distances
    seats.sort()
    students.sort()

    # Compute abs distance between seats and students
    moves = 0
    for i in range(N):
        moves += abs(seats[i] - students[i])
    return moves

# Testing the solution
assert minMovesToSeat([3,1,5], [2,7,4]) == 4
assert minMovesToSeat([2,2,6,6], [1,3,2,6]) == 4

LC Medium: Minimum increment to make array unique
https://leetcode.com/problems/minimum-increment-to-make-array-unique

In [12]:
def minIncrementForUnique(nums: list[int]) -> int:
    # Count freq & unique values present
    counts: dict[int, int] = dict()
    unique: set[int] = set()
    for n in nums:
        counts[n] = counts.get(n, 0) + 1
        unique.add(n)

    # If count of any number is greater than 1, shift right
    moves = 0
    while unique:
        key = unique.pop()
        count = counts.pop(key)
        if count > 1:
            shift = count - 1
            moves += shift
            counts[key + 1] = counts.get(key + 1, 0) + shift
            unique.add(key + 1)

    return moves

# Testing the solution
assert minIncrementForUnique([1,2,2]) == 1
assert minIncrementForUnique([3,2,1,2,1,7]) == 6
assert minIncrementForUnique([1,2,2,5,5]) == 2

Maximum Gap
https://leetcode.com/problems/maximum-gap/

In [13]:
# https://leetcode.com/problems/maximum-gap/submissions/1288173457/
def maximumGap(nums: list[int]) -> int:
    """
    1. Maximum gap is always greater than or equal to ceil((max - min) / N)
        - Divide N nums into N - 1 groups
        - If evenly distributed, each elements would be in a different bucket (1 element in each bucket)
        - If unevenly distributed, two or more buckets would have multiple elements. Some buckets may not have any elements - max gap could occur betweeen left and right of the missing buckets
    2. Consequently we only need to store the min and max of each bucket
    """
    N, high, low = len(nums), max(nums), min(nums)
    bucket_size = math.ceil((high - low) / N)
    buckets: dict[int, tuple[int, int]] = {}

    # Edge case
    if bucket_size == 0:
        return 0

    # Group the numbers into buckets
    for n in nums:
        index = n // bucket_size
        min_, max_ = buckets.get(index, ((index + 1) * bucket_size, (index - 1) * bucket_size))
        min_, max_ = min(min_, n), max(max_, n)
        buckets[index] = (min_, max_)

    # Iterate through each and find the max gap
    curr, end = low // bucket_size, high // bucket_size
    max_gap = 0
    while curr < end:
        next_ = curr + 1
        while next_ not in buckets:
            next_ += 1
        curr, max_gap = next_, max(max_gap, buckets[next_][0] - buckets[curr][1])

    return max_gap

# Testing the solution
assert maximumGap([3,14,15,83,6,4,19,20,40]) == 43
assert maximumGap([3,6,9,1]) == 3
assert maximumGap([10]) == 0

LC Virtual - 5: Weekly Contest 6
https://leetcode.com/contest/leetcode-weekly-contest-6/

In [14]:
# Q1: Sum of left leaves
def sumOfLeftLeaves(root: BinaryTreeNode) -> int:
    def backtrack(curr: BinaryTreeNode) -> tuple[int, int]:
        if not curr:
            return 0, -1
        else:
            left_sum, left_type = backtrack(curr.left)
            right_sum, right_type = backtrack(curr.right)
            curr_type, total = 0 if left_type == -1 and right_type == -1 else 1, 0
            if curr_type == 0:
                total += curr.val
            if left_type != -1:
                total += left_sum
            if right_type >= 1:
                total += right_sum
            result = total, curr_type
            return result

    return backtrack(root)[0] if root.left or root.right else 0

# Testing the solution
assert sumOfLeftLeaves(BinaryTreeNode.from_array([3,9,20,None,None,15,7])) == 24
assert sumOfLeftLeaves(BinaryTreeNode.from_array([1])) == 0

In [15]:
# Q2: Convert to hexadecimal
def toHex(num: int) -> str:
    # If negative, find hex for its 2's compliment
    if num < 0:
        bit_length = 32
        num &= (2 << bit_length - 1) - 1

    # Use division method
    division_result: list[int] = []
    while num >= 16:
        num, rem = num // 16, num % 16
        division_result.append(rem)
    division_result.append(num)
    division_result.reverse()

    # Convert to hexadecimal string
    mapping = {i: str(i) for i in range(0, 10)} | {i: chr(ord('a') + i - 10) for i in range(10, 16)}
    return ''.join(map(lambda x: mapping[x], division_result))

# Testing the solution
assert toHex(26) == "1a"
assert toHex(-1) == "ffffffff"

In [16]:
# Q3: Queue Reconstruction by Height
def reconstructQueue(people: list[list[int]]) -> list[list[int]]:
    N = len(people)

    # Sort to find the relative height ordering
    people.sort()

    # Iterate from shortest to tallest, keeping track of number of person with height equal to curr
    eq_front = []
    prev, eq_count = -1, 0
    for i in range(N):
        if people[i][0] == prev:
            eq_count += 1
        else:
            eq_count = 0
        prev = people[i][0]
        eq_front.append(eq_count)

    # Insert into queue in correct order
    result: list[list[int]] = []
    while eq_front:
        actual, (height, expected) = eq_front.pop(), people.pop()
        if expected == actual:
            result.append([height, expected])
        else:
            temp: list[list[int]] = []
            for i in range(expected - actual):
                temp.append(result.pop())
            result.append([height, expected])
            while temp:
                result.append(temp.pop())

    result.reverse()
    return result

# Testing the solution
assert reconstructQueue([[7,0],[4,4],[7,1],[5,0],[6,1],[5,2]]) == [[5,0],[7,0],[5,2],[6,1],[4,4],[7,1]]
assert reconstructQueue([[6,0],[5,0],[4,0],[3,2],[2,2],[1,4]]) == [[4,0],[5,0],[2,2],[3,2],[1,4],[6,0]]

Trapping Rain water - II (Upsolving)
https://leetcode.com/problems/trapping-rain-water-ii/

In [17]:
# Q4: Trapping Rain water - II
# https://leetcode.com/problems/trapping-rain-water-ii/solutions/1138028/python3-visualization-bfs-solution-with-explanation
def trapRainWater(heightMap: list[list[int]]) -> int:
    N, M = len(heightMap), len(heightMap[0])
    visited: set[tuple[int, int]] = set()

    # Add boundary cells to heap, heap: Height, i, j
    heap: list[tuple[int, int, int]] = []
    for i in range(N):
        for j in range(M):
            if i in (0, N - 1) or j in (0, M - 1):
                heapq.heappush(heap, (heightMap[i][j], i, j))
                visited.add((i, j))

    level = volume = 0
    while heap:
        height, x, y = heapq.heappop(heap)
        level = max(level, height)
        for i, j in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
            if 0 <= i < N and 0 <= j < M and (i, j) not in visited:
                next_height = heightMap[i][j]
                if next_height < level:
                    volume += level - next_height
                visited.add((i, j))
                heapq.heappush(heap, (next_height, i, j))

    return volume

# Testing the solution
assert trapRainWater([[1,4,3,1,3,2],[3,2,1,3,2,4],[2,3,3,2,3,1]]) == 4
assert trapRainWater([[3,3,3,3,3],[3,2,2,2,3],[3,2,1,2,3],[3,2,2,2,3],[3,3,3,3,3]]) == 10
assert trapRainWater([[12,13,1,12],[13,4,13,12],[13,8,10,12],[12,13,12,12],[13,13,13,13]]) == 14
assert trapRainWater([[5,5,5,1],[5,1,1,5],[5,1,5,5],[5,2,5,8]]) == 3
assert trapRainWater([[14,17,18,16,14,16],[17,3,10,2,3,8],[11,10,4,7,1,7],[13,7,2,9,8,10],[13,1,3,4,8,6],[20,3,3,9,10,8]]) == 25

https://leetcode.com/contest/leetcode-weekly-contest-8/problems/pacific-atlantic-water-flow/

In [18]:
def pacificAtlantic(heights: list[list[int]]) -> set[tuple[int, int]]:
    def DFS(to_visit: list[tuple[int, int]]) -> set[tuple[int, int]]:
        visited: set[tuple[int, int]] = set()
        while to_visit:
            x, y = to_visit.pop()
            visited.add((x, y))
            for i, j in [(x, y + 1), (x, y - 1), (x + 1, y), (x - 1, y)]:
                if 0 <= i < N and 0 <= j < M and (i, j) not in visited and heights[i][j] >= heights[x][y]:
                    to_visit.append((i, j))

        return visited

    N, M = len(heights), len(heights[0])

    pacific: set[tuple[int, int]] = DFS([(0, i) for i in range(M)] + [(i, 0) for i in range(N)])
    atlantic: set[tuple[int, int]] = DFS([(N - 1, i) for i in range(M)] + [(i, M - 1) for i in range(N)])

    return set.intersection(pacific, atlantic)

# Testing the soluiton
pacificAtlantic([[1,2,2,3,5],[3,2,3,4,4],[2,4,5,3,1],[6,7,1,4,5],[5,1,1,2,4]])

{(0, 4), (1, 3), (1, 4), (2, 2), (3, 0), (3, 1), (4, 0)}

In [19]:
# https://leetcode.com/contest/leetcode-weekly-contest-8/problems/sentence-screen-fitting/
def sentenceScreenFitting(sentence: list[str], rows: int, cols: int) -> int:
    n_words = len(sentence)
    word_length: list[int] = list(map(len, sentence))
    word_idx = n_sentences = 0

    row_pending = rows
    while row_pending > 0:
        col_pending = cols
        while col_pending > 0:

            if word_length[word_idx] <= col_pending:
                col_pending -= word_length[word_idx]
                word_idx += 1

                if col_pending > 0:
                    col_pending -= 1

            else:
                col_pending = 0

            if word_idx == n_words:
                word_idx = 0
                n_sentences += 1

        row_pending -= 1

    return n_sentences

# Testing the solution
assert sentenceScreenFitting(["hello","world"], 2, 8) == 1
assert sentenceScreenFitting(["a", "bcd", "e"], 3, 6) == 2
assert sentenceScreenFitting(["i","had","apple","pie"], 4, 5) == 1

Codeforces: https://codeforces.com/problemset/problem/1843/C

In [20]:
def sumBinaryTree(n: int) -> int:
    if n == 1:
        return 1
    elif n % 2 == 0:
        return n + sumBinaryTree(n // 2)
    else:
        return n + sumBinaryTree((n - 1) // 2)

# Testing the solution
assert sumBinaryTree(37) == 71
assert sumBinaryTree(10000000000000000) == 19999999999999980
assert sumBinaryTree(15) == 26

In [21]:
def insertionInBST(curr: BinaryTreeNode, val: int):
    if not curr:
        curr = BinaryTreeNode(val)
    elif curr.val < val:
        curr.right = insertionInBST(curr.right, val)
    else:
        curr.left = insertionInBST(curr.left, val)
    return curr

# Testing the solution
assert insertionInBST(BinaryTreeNode.from_array([10, 2, 12, 1, 3, None, 13]), 11).to_list() == [10, 2, 12, 1, 3, 11, 13]

King Escape: https://codeforces.com/problemset/problem/1033/A

In [22]:
def kingEscape(N: int, queen: tuple[int, int], king: tuple[int, int], target: tuple[int, int]) -> bool:
    minX, maxX = min(king[0], target[0]), max(king[0], target[0])
    minY, maxY = min(king[1], target[1]), max(king[1], target[1])
    return not (minX < queen[0] < maxX or minY < queen[1] < maxY)

# Testing the solution
assert kingEscape(8, (4, 4), (1, 3), (3, 1)) == True
assert kingEscape(8, (4, 4), (2, 3), (1, 6)) == False
assert kingEscape(8, (3, 5), (1, 2), (6, 1)) == False

LC Medium: https://leetcode.com/problems/most-profit-assigning-work/submissions/1292541583

In [23]:
def maxProfitAssignment(difficulty: list[int], profit: list[int], worker: list[int]) -> int:
    # Sort workers by their capacity
    worker.sort()

    # Sort jobs available based on their difficuly
    jobs: list[tuple[int, int]] = sorted(zip(difficulty, profit))

    # Use a two pointer approach - iterate through jobs adding to our maxHeap until diff cross what curr worker can handle
    heap: list[int] = []
    maxProfit = i = 0
    for max_difficulty in worker:
        while i < len(jobs) and jobs[i][0] <= max_difficulty:
            heapq.heappush(heap, -jobs[i][1])
            i += 1
        maxProfit += -heap[0] if heap else 0

    return maxProfit

# Testing the solution
assert maxProfitAssignment([2,4,6,8,10], [10,20,30,40,50], [4,5,6,7]) == 100
assert maxProfitAssignment([85,47,57], [24,66,99], [40,25,25]) == 0
assert maxProfitAssignment([13,37,58], [4,90,96], [34,73,45]) == 190
assert maxProfitAssignment([68,35,52,47,86], [67,17,1,81,3], [92,10,85,84,82]) == 324

https://codeforces.com/problemset/problem/1829/E

In [24]:
class DisjointSetLakes:
    def __init__(self, N: int, M: int, grid: list[list[int]]):
        self.N, self.M = N, M
        self.grid = grid
        self.parent: dict[tuple[int, int], tuple[int, int]] = dict()
        self.size: dict[tuple[int, int], int] = dict()
        self.max_volume = 0
        self.iterate_grid()

    def is_valid(self, i: int, j: int) -> bool:
        return 0 <= i < self.N and 0 <= j < self.M

    def iterate_grid(self) -> None:
        for i in range(self.N):
            for j in range(self.M):
                if self.grid[i][j] > 0:
                    top, left = (i - 1, j), (i, j - 1)
                    if self.is_valid(*top) and self.grid[top[0]][top[1]] > 0:
                        self.unionBySize((i, j), top)
                    if self.is_valid(*left) and self.grid[left[0]][left[1]] > 0:
                        self.unionBySize((i, j), left)
                    self.max_volume = max(self.max_volume, self.size[self.findUltimateParent((i, j))])

    def unionBySize(self, c1: tuple[int, int], c2: tuple[int, int]) -> None:
        up1, up2 = self.findUltimateParent(c1), self.findUltimateParent(c2)
        s1, s2 = self.size[up1], self.size[up2]
        if up1 == up2:
            return
        elif s1 > s2:
            self.parent[up2] = up1
            self.size[up1] += self.size[up2]
        else:
            self.parent[up1] = up2
            self.size[up2] += self.size[up1]

        # Update the max volume
        self.max_volume = max(self.max_volume, self.size[up1], self.size[up2])

    def findUltimateParent(self, cell: tuple[int, int]) -> tuple[int, int]:
        if cell not in self.parent:
            self.parent[cell] = cell
            self.size[cell] = self.grid[cell[0]][cell[1]]

        if cell == self.parent[cell]:
            return cell
        else:
            self.parent[cell] = self.findUltimateParent(self.parent[cell])
            return self.parent[cell]

# Testing the solution
assert DisjointSetLakes(3, 3, [[1,2,0],[3,4,0],[0,0,5]]).max_volume == 10
assert DisjointSetLakes(1, 1, [[9]]).max_volume == 9
assert DisjointSetLakes(3, 3, [[0,1,1],[1,0,1],[1,1,1]]).max_volume == 7
assert DisjointSetLakes(5, 5, [[1,1,1,1,1],[1,0,0,0,1],[1,0,5,0,1],[1,0,0,0,1],[1,1,1,1,1]]).max_volume == 16